In [2]:
# Import Packages
import numpy as np
import cvxpy as cp
import pandas as pd
import matplotlib.pyplot as plt
# Import functions
from my_functions import *

In [3]:
df_new = pd.read_csv('data/complete_data.csv')
# total number of cars 
N = len(df_new)
# value of cars varies
profit = df_new['Profit']
# size of the cars varies
size = df_new["size"]
# total size of all cars
total_size = np.sum(size)
# parking lot capacity
parkCapacity = 2*total_size/3

# Buying price for each car
buy_price = df_new["Buying Price"]
# total amount of buying cost
total_price = np.sum(buy_price)
# total budget the dealership has
dealBudget = total_price/4

fee_electric = 300
fee_imported = 1000

In [ ]:
# Outputs
infoPage(N, profit, total_size, buy_price, parkCapacity, dealBudget)
print("---------------------------------------------------------------")
print("DealershipDecisionOptimizerSystem(DDOS) is working....")
# Solution

#adjusting array sizes to accommodate for penalty fee dummy variables
x = cp.Variable(N+2,integer=True)
s_wdummy=np.append(size, [0, 0])
b_wdummy=np.append(buy_price, [0, 0])
profit_wdummy=np.append(profit, [-fee_electric, -fee_imported])

#creating ILP set up with space and budget constraints
obj = cp.Maximize(cp.sum(cp.multiply(profit_wdummy,x)))
constraints = [cp.sum(cp.multiply(s_wdummy,x)) <= parkCapacity,
               cp.sum(cp.multiply(b_wdummy,x)) <= dealBudget,
               x <= 1, x >= 0]

#adding constraints for dummy e>=x_j, e<= sum(x_j) for all electric cars j and i>=x_k, i<=sum(x_k) for all imported cars k
e_sum=np.zeros(N+2)
i_sum=np.zeros(N+2)

df_new = df_new.reset_index(drop=True)
for k in range(len(df_new)):
    if ((df_new.loc[k, 'fuel_type'].lower() == 'Hybrid') or (df_new.loc[k, 'fuel_type'].lower() == 'Plug-In Hybrid')):
        constraints.append(x[k] >= x[N-2])
        e_sum[k]=1
    elif df_new.loc[k, 'imported'] == '1':
        constraints.append(x[k] >= x[N-1])
        i_sum[k]=1

constraints.append(cp.sum(cp.multiply(e_sum,x)) <= x[N-2])
constraints.append(cp.sum(cp.multiply(i_sum,x)) <= x[N-1])
	
prob = cp.Problem(obj,constraints)
prob.solve()
print("---------------------------------------------------------------")
print(np.sum(x.value), "cars sold")
print("Profit: $", profit_wdummy@x.value)
print("Actual Size of cars sold: ", s_wdummy@x.value, "m^2")
print("Actual budget spent: $", b_wdummy@x.value)